In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
from skimage.transform import resize
from random import shuffle
from keras.applications.resnet50 import ResNet50
from keras.layers import Input,Flatten,Dense,Dropout
from keras.models import Model 
from keras.optimizers import SGD


Using TensorFlow backend.


In [3]:
#读取图片
list_paths = []
for subdir, dirs, files in os.walk(r"H:\DeepLearning_DataSet\Kaggle_IEEE"):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file
        list_paths.append(filepath)
    print(list_paths)

list_train = [filepath for filepath in list_paths if "train_manip\\" in filepath]
shuffle(list_train)
list_test = [filepath for filepath in list_paths if "test\\" in filepath]

# list_train = list_train
# list_test = list_test
index = [os.path.basename(filepath) for filepath in list_test]

list_classes = list(set([os.path.dirname(filepath).split(os.sep)[-1] for filepath in list_paths if "train" in filepath]))

# list_classes = ['Sony-NEX-7',
                #  'Motorola-X',
                #  'HTC-1-M7',
                #  'Samsung-Galaxy-Note3',
                #  'Motorola-Droid-Maxx',
                #  'iPhone-4s',
                #  'iPhone-6',
                #  'LG-Nexus-5x',
                #  'Samsung-Galaxy-S4',
                #  'Motorola-Nexus-6']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#图片预处理
ishape = 224

def get_class_from_path(filepath):
    return os.path.dirname(filepath).split(os.sep)[-1]

def read_and_resize(filepath):
    im_array = np.array(Image.open((filepath)), dtype="uint8")
    pil_im = Image.fromarray(im_array)
    new_array = np.array(pil_im.resize((ishape, ishape)))
    return new_array/255

def label_transform(labels):
    labels = pd.get_dummies(pd.Series(labels))
    label_index = labels.columns.values

    return labels, label_index

list_classes



In [5]:
#图片数据化处理
tempIndex = len(list_train) // 2

X_train = np.array([read_and_resize(filepath) for filepath in list_train[0:tempIndex]])
labels = [get_class_from_path(filepath) for filepath in list_train[0:tempIndex]]
y, label_index = label_transform(labels) #二值化
y = np.array(y)

X_test = np.array([read_and_resize(filepath) for filepath in list_test])


In [5]:
#存储图片数据
np.save(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\X_train.npy",X_train)
np.save(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\y.npy",y)
np.save(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\X_test.npy",X_test)

In [6]:
#读取图片数据
X_train = np.load(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\X_train.npy")
y_train = np.load(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\y.npy")
X_test = np.load(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\X_test.npy")


In [8]:
def generate_arrays_from_file(batch_size):
    while 1:
        count = 0
        X = []
        Y = []
        for x1,x2,y1,y2 in zip(X_train1,X_train2,y_train1,y_train2):    
            X.append(x1)
            X.append(x2)
            Y.append(y1)
            Y.append(y2)
            count += 1
            if count == batch_size:
                count = 0
                yield (np.array(X),np.array(Y))
                X = []
                Y = []

array([[[[ 0.68627451,  0.68627451,  0.65490196],
         [ 0.69803922,  0.69803922,  0.66666667],
         [ 0.68235294,  0.68235294,  0.65098039],
         ..., 
         [ 0.59215686,  0.57647059,  0.53333333],
         [ 0.62745098,  0.61176471,  0.56862745],
         [ 0.61568627,  0.6       ,  0.55686275]],

        [[ 0.67058824,  0.67058824,  0.63921569],
         [ 0.70196078,  0.70196078,  0.67058824],
         [ 0.66666667,  0.66666667,  0.63529412],
         ..., 
         [ 0.67843137,  0.6627451 ,  0.61960784],
         [ 0.70588235,  0.69019608,  0.64705882],
         [ 0.68235294,  0.66666667,  0.62352941]],

        [[ 0.63921569,  0.63921569,  0.60784314],
         [ 0.63921569,  0.63921569,  0.60784314],
         [ 0.65882353,  0.65882353,  0.62745098],
         ..., 
         [ 0.5372549 ,  0.52156863,  0.47843137],
         [ 0.6627451 ,  0.64705882,  0.60392157],
         [ 0.70196078,  0.68627451,  0.64313725]],

        ..., 
        [[ 0.76078431,  0.73333333,

In [29]:
#Resnet50

model_Res = ResNet50(include_top=False,#是否含输入层
                  weights='imagenet',
                  input_shape=(ishape,ishape,3)
                  )
 
for layer in model_Res.layers:                 
    layer.trainable = False

model = Flatten(name='flatten')(model_Res.output)
model = Dense(32,activation='relu')(model)
# model = Dropout(0.5)(model)
# model = Dense(256,activation='relu')(model)
model = Dense(10,activation='softmax')(model)
model_Res_img = Model(model_Res.input,model,name='Res')

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
sgd = SGD(lr = 0.003,decay= 1e-5)
model_Res_img.compile(loss='categorical_crossentropy',
                      optimizer=sgd,
                      metrics=['accuracy'])
early = EarlyStopping(monitor="val_acc", mode="max", patience=3)
# model_Res_img.fit(X_train,y,validation_split=0.2,epochs=100,batch_size=80,callbacks=[early]) #callbacks=[early])
model_Res_img.fit_generator(generate_arrays_from_file(80),samples_per_epoch=17600,validation_split=0.2,max_q_size=1000,callbacks=[early],verbose=1)
# model_Res_img.summary()


8800/8800 [==============================] - 119s 14ms/step - loss: 0.2074 - acc: 0.9489 - val_loss: 0.4360 - val_acc: 0.8609


Epoch 83/100


  50/8800 [..............................] - ETA: 1:29 - loss: 0.1387 - acc: 0.9800

 100/8800 [..............................] - ETA: 1:29 - loss: 0.2049 - acc: 0.9400

 150/8800 [..............................] - ETA: 1:28 - loss: 0.2150 - acc: 0.9400

...

8600/8800 [============================>.] - ETA: 2s - loss: 0.1731 - acc: 0.9600

8650/8800 [============================>.] - ETA: 1s - loss: 0.1732 - acc: 0.9601

8700/8800 [============================>.] - ETA: 1s - loss: 0.1734 - acc: 0.9597

8750/8800 [============================>.] - ETA: 0s - loss: 0.1737 - acc: 0.9595

8800/8800 [==============================] - 113s 13ms/step - loss: 0.1735 - acc: 0.9595 - val_loss: 0.4087 - val_acc: 0.8709


In [34]:
#预测
predicts = model_Res_img.predict(X_test)
predicts = np.argmax(predicts, axis=1)
predicts = [label_index[p] for p in predicts]

df = pd.DataFrame(columns=['fname', 'camera'])
df['fname'] = index
df['camera'] = predicts


In [36]:
df.tail()

,fname,camera
2635,img_fee2c32_unalt.tif,Motorola-Droid-Maxx
2636,img_fefdce8_manip.tif,Motorola-X
2637,img_ff225ec_manip.tif,Motorola-Nexus-6
2638,img_ff56ac2_unalt.tif,Sony-NEX-7
2639,img_ffaeda7_unalt.tif,iPhone-4s


In [38]:
#保存结果
df.to_csv(r"H:\PycharmProjects\MyKaggle\Camera_Model_Identification\result_res50_6.csv", index=False)